In [11]:
import gp_module
import pandas as pd

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna 
from ydata_profiling import ProfileReport
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error , mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor
import joblib
import warnings
from xgboost import XGBRegressor
from gplearn.genetic import SymbolicRegressor
import numpy as np
import random
import math
from typing import List, Callable
import json 
from catboost import CatBoostRegressor
#from lightgbm import LGBMRegressor
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
import gp_module
import numpy as np
import random
import math
from typing import List, Callable
X_train = pd.read_csv('../Data/dataset/X_train.csv')
features_number = X_train.shape[1]
X_train = pd.read_csv('../Data/dataset/X_train.csv').values
y_train = pd.read_csv('../Data/dataset/Y_train.csv').values.ravel()
X_val = pd.read_csv('../Data/dataset/X_val.csv').values
y_val = pd.read_csv('../Data/dataset/Y_val.csv').values.ravel()
X_test = pd.read_csv('../Data/dataset/X_test.csv').values
y_test = pd.read_csv('../Data/dataset/Y_test.csv').values.ravel()




In [ ]:
class Symbol:
    def __init__(self, is_function: bool, arity: int, name: str, value: float = 0.0):
        self.is_function = is_function
        self.arity = arity
        self.name = name
        self.value = value

class Node:
    def __init__(self, root: Symbol, children: List['Node'] = None):
        self.root = root
        self.children = children or []

class Individual:
    def __init__(self, root: Node):
        self.root = root
        self.fitness = float('inf')

class SymbolicRegressor:
    def __init__(self, params: dict):
        self.params = params
        self.symbols = self.create_symbols()
        self.population = []
        self.best_individual = None

    def create_symbols(self):
        symbols = [
            Symbol(True, 2, "+"),
            Symbol(True, 2, "-"),
            Symbol(True, 2, "*"),
            Symbol(True, 2, "/")
        ]
        symbols.extend([Symbol(False, 0, f"x{i}") for i in range(self.params['num_variables'])])
        symbols.extend([Symbol(False, 0, str(random.uniform(self.params['min_constant'], self.params['max_constant']))) 
                        for _ in range(self.params['num_constants'])])
        return symbols

    def create_tree(self, depth: int, max_depth: int) -> Node:
        if depth == max_depth or (depth > 0 and random.random() < 0.5):
            return Node(random.choice([s for s in self.symbols if not s.is_function]))
        
        root = random.choice([s for s in self.symbols if s.is_function])
        children = [self.create_tree(depth + 1, max_depth) for _ in range(root.arity)]
        return Node(root, children)

    def create_population(self):
        self.population = [Individual(self.create_tree(0, self.params['max_depth'])) 
                           for _ in range(self.params['population_size'])]

    def evaluate(self, individual: Individual, X: np.ndarray, y: np.ndarray) -> float:
        predictions = np.array([self.eval_tree(individual.root, x) for x in X])
        return np.sqrt(np.mean((predictions - y) ** 2))

    def eval_tree(self, node: Node, x: np.ndarray) -> float:
        if not node.root.is_function:
            return float(node.root.name) if node.root.name[0] != 'x' else x[int(node.root.name[1:])]
        
        if len(node.children) == 0:
            return 0  # Return a default value if there are no children
        
        if len(node.children) == 1:
            return self.eval_tree(node.children[0], x)  # Return the value of the single child
        
        left = self.eval_tree(node.children[0], x)
        right = self.eval_tree(node.children[1], x) if len(node.children) > 1 else 0

        if node.root.name == '+': return left + right
        if node.root.name == '-': return left - right
        if node.root.name == '*': return left * right
        if node.root.name == '/': return left / right if right != 0 else 1

    def tournament_selection(self) -> Individual:
        tournament = random.sample(self.population, self.params['tournament_size'])
        return min(tournament, key=lambda ind: ind.fitness)

    def crossover(self, parent1: Individual, parent2: Individual) -> Individual:
        def swap_subtree(node1: Node, node2: Node) -> Node:
            if random.random() < 0.5:
                return Node(node2.root, node2.children)
            if not node1.root.is_function:
                return Node(node1.root)
            new_children = [swap_subtree(c1, c2) for c1, c2 in zip(node1.children, node2.children)]
            return Node(node1.root, new_children)
        
        new_root = swap_subtree(parent1.root, parent2.root)
        return Individual(new_root)

    def mutation(self, individual: Individual) -> Individual:
        def mutate_node(node: Node) -> Node:
            if random.random() < self.params['mutation_rate']:
                return self.create_tree(0, self.params['max_depth'])
            if not node.root.is_function:
                return Node(node.root)
            new_children = [mutate_node(child) for child in node.children]
            return Node(node.root, new_children)
        
        new_root = mutate_node(individual.root)
        return Individual(new_root)

    def fit(self, X: np.ndarray, y: np.ndarray):
        self.create_population()
        for generation in range(self.params['num_generations']):
            for ind in self.population:
                ind.fitness = self.evaluate(ind, X, y)
            
            self.best_individual = min(self.population, key=lambda ind: ind.fitness)
            new_population = [self.best_individual]  # Elitism

            while len(new_population) < self.params['population_size']:
                if random.random() < self.params['crossover_rate']:
                    parent1, parent2 = self.tournament_selection(), self.tournament_selection()
                    offspring = self.crossover(parent1, parent2)
                else:
                    offspring = self.mutation(self.tournament_selection())
                new_population.append(offspring)

            self.population = new_population

            if generation % 10 == 0:  # Print progress every 10 generations
                print(f"Generation {generation}: Best Fitness = {self.best_individual.fitness}")

        return self.best_individual.fitness

    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.array([self.eval_tree(self.best_individual.root, x) for x in X])

# Example usage

    

def objective(trial):
    params = {
        'population_size': trial.suggest_int('population_size', 50, 300),
        'num_generations': trial.suggest_int('num_generations', 50, 200),
        'tournament_size': trial.suggest_int('tournament_size', 2, 20),
        'crossover_rate': trial.suggest_float('crossover_rate', 0.1, 1.0),
        'mutation_rate': trial.suggest_float('mutation_rate', 0.01, 0.5),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_variables': features_number,  
        'num_constants': trial.suggest_int('num_constants', 5, 30),
        'min_constant': trial.suggest_float('min_constant', -10, -1),
        'max_constant': trial.suggest_float('max_constant', 1, 10)
    }
    print(f"Trying parameters: {params}")

    regressor = SymbolicRegressor(params)

    regressor.fit(X_train, y_train)

    y_pred_val = regressor.predict(X_val)

    mse_val = np.mean((y_val - y_pred_val) ** 2)

    return mse_val  

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  #

print("Best parameters:", study.best_params)

best_params = study.best_params
best_regressor = SymbolicRegressor(best_params)
best_regressor.fit(X_train, y_train)


y_pred_test = best_regressor.predict(X_test)
test_mse = np.mean((y_test - y_pred_test) ** 2)
print(f"Test MSE: {test_mse}")
